In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Wgranie danych**

In [ ]:
column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'PRICE']
boston = pd.read_csv('/kaggle/input/boston-house-prices/housing.csv', header=None, delimiter=r"\s+", names=column_names)

* CRIM - wskaźnik przestępczości na mieszkańca według miast,
* ZN - udział terenów mieszkalnych przeznaczonych pod działki powyżej 25 tys. Mkw.,
* INDUS - udział niedetalicznych akrów biznesowych przypadających na miasto,
* CHAS - zmienna zastępcza Charles River (1 jeśli trakt ogranicza rzekę; 0 w przeciwnym razie),
* NOX - stężenie tlenków azotu (cząstki na 10 mln),
* RM - średnia liczba izb w mieszkaniu,
* AGE - odsetek lokali okupowanych przez właścicieli zbudowanych przed 1940 rokiem, 
* DIS - ważone odległości do pięciu centrów pracy w Bostonie,
* RAD - wskaźnik dostępności do autostrad radialnych,
* TAX - pełnowartościowa stawka podatku od nieruchomości na 10000 USD, 
* PTRATIO - stosunek liczby uczniów do nauczycieli według miast,
* B - 1000 (Bk - 0,63) ^ 2 gdzie Bk to odsetek osób czarnoskórych w mieście, 
* LSTAT – odsetek osób o niższym statusie,
* PRICE- Mediana wartości domów zajmowanych przez właścicieli w tysiącach dolarów

In [ ]:
boston.head()

In [ ]:
boston.shape

Dane zawirają 14 zmiennych i 506 rekordów

In [ ]:
boston.isnull().sum()

Nie zawierają rzadnych braków

In [ ]:
corr = boston.corr()
corr.shape

In [ ]:
boston.hist(figsize=(10, 10), bins=20)
plt.show()

In [ ]:
boston.describe()

In [ ]:
plt.figure(figsize=(20,10))
plt.boxplot(boston)
plt.show()

**Macierz korelacji**

In [ ]:
plt.figure(figsize=(20, 10))
sns.heatmap(corr,  annot=True, cmap='twilight_r')

**Dzielimy nasze dane na dwa zbiory:**

* X – zmienne bez zmiennej celu, zbiór posłuży nam do uczenia naszego modelu
* y – wartości do uczenia modelu zmiennej celu

Zdecydowałam się na tworzenie modelu z wykorzystaniem tylko zmiennych ZN, INDUS, RM, AGE, DIS, TAX, PTRATIO, LSTAT.
Usunełam zmienne CHAS i NOX ze względu na wysoką korelację.

In [ ]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
column_sels = ['ZN', 'INDUS', 'RM', 'AGE', 'DIS', 'TAX', 'PTRATIO', 'LSTAT']
X = boston.loc[:,column_sels]
y = boston['PRICE']
fig, axs = plt.subplots(ncols=4, nrows=2, figsize=(20, 10))
index = 0
axs = axs.flatten()
for i, k in enumerate(column_sels):
    sns.regplot(y=y, x=X[k], ax=axs[i])
plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=5.0)

Po obejrzeniu wykresów zdecydowałam się na usunięcie jeszcze zmiennej B, RAD, CRIM

**Model**
* Regresja liniowa

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 4)

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)

Parametry modelu

In [ ]:
coeffcients = pd.DataFrame([X_train.columns,lr.coef_]).T
coeffcients = coeffcients.rename(columns={0: 'Zmienna', 1: 'Współczynnik kierunkowy'})
coeffcients

In [ ]:
print('Wyraz wolny: ', lr.intercept_)

In [ ]:
y_pred = lr.predict(X_train)

**Wyniki działania modelu**

* MAE - Średni błąd bezwzględny reprezentuje średnią bezwzględnej różnicy między rzeczywistymi i przewidywanymi wartościami w zbiorze danych. Mierzy średnią reszt w zbiorze danych.
* MSE - Błąd średniokwadratowy reprezentuje średnią kwadratów różnicy między oryginalnymi i przewidywanymi wartościami w zestawie danych. Mierzy wariancję reszt.
* RMSE - Podstawowy błąd średniokwadratowy to pierwiastek kwadratowy błędu średniokwadratowego. Mierzy odchylenie standardowe reszt.
* R^2 - Współczynnik determinacji lub R-kwadrat reprezentuje proporcję wariancji zmiennej zależnej, która jest wyjaśniona przez model regresji liniowej. Jest to wynik bez skali, tj. Niezależnie od tego, czy wartości są małe czy duże, wartość R kwadrat będzie mniejsza niż jeden.
* Adjusted R^2 - Skorygowany R kwadrat jest zmodyfikowaną wersją R-kwadrat i jest dostosowany do liczby niezależnych zmiennych w modelu i zawsze będzie mniejszy lub równy R².




In [ ]:
print('R^2:',metrics.r2_score(y_train, y_pred))
print('Adjusted R^2:',1 - (1-metrics.r2_score(y_train, y_pred))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1))
print('MAE:',metrics.mean_absolute_error(y_train, y_pred))
print('MSE:',metrics.mean_squared_error(y_train, y_pred))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_train, y_pred)))

In [ ]:
plt.scatter(y_train, y_pred)
plt.xlabel("Prices")
plt.ylabel("Predicted prices")
plt.title("Prices do Predicted prices")
plt.show()

In [ ]:
y_test_pred = lr.predict(X_test)

In [ ]:
acc_linreg = metrics.r2_score(y_test, y_test_pred)
print('R^2:', acc_linreg)
print('Adjusted R^2:',1 - (1-metrics.r2_score(y_test, y_test_pred))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1))
print('MAE:',metrics.mean_absolute_error(y_test, y_test_pred))
print('MSE:',metrics.mean_squared_error(y_test, y_test_pred))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)))

In [ ]:
lr.score(X_test,y_test)

**Krzywe uczenia**

In [ ]:
train_errors = []
test_error = []

In [ ]:
from sklearn.metrics import mean_squared_error
def plot_learning_curve(model,X,y):
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 4)

    for m in range(1, len(X_train)):
        model.fit(X_train[:m],y_train[:m])
        y_train_pred = model.predict(X_train[:m])
        y_test_pred = model.predict(X_test[:m])
        train_errors.append(metrics.r2_score(y_train[:m], y_train_pred))
        test_error.append(metrics.r2_score(y_test[:m], y_test_pred))
        
    plt.plot(np.sqrt(train_errors),"r-+",linewidth=2,label="train")
    plt.plot(np.sqrt(test_error),"b-",linewidth=3,label="test")
    plt.xlabel('Rozmiar zestawu uczącego')
    plt.ylabel('Błąd R^2')
    plt.title('Krzywa uczenia')
    plt.legend() 
    plt.show()


In [ ]:
lr = LinearRegression()
plot_learning_curve(lr,X_train,y_train)   

**Regularyzowane modele liniowe**

Standaryzacja danych

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)

Regresja Lasso i regresja grzbietowa

In [ ]:
models = []
models.append( ('Ridge', Ridge()) )
models.append( ('Lasso', Lasso()) )

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
import joblib
for name, model in models:
 pipelined_model = Pipeline([('minmax', MinMaxScaler()),('pca', PCA(n_components = 3)),(name, model)])
 pipelined_model.fit(rescaledX, y_train)
 y_hat = pipelined_model.predict(X_test)
 RMSE = np.sqrt(mean_squared_error(y_test, y_hat))
 print('Model: ', name, ' | RMSE: ', RMSE)
 print('----------------')
 joblib.dump(pipelined_model, '{}_model.pkl'.format(name))

Widzimy, że dla danych po standaryzacji lepszy wynik dostajemy przy regresji lasso.

**Predyktor maszyny wektorów nośnych**

In [ ]:
 from sklearn.svm import SVC
 from sklearn.preprocessing import StandardScaler
 from sklearn.datasets import make_classification
 from sklearn.model_selection import train_test_split
 from sklearn.pipeline import Pipeline

**Model SVC**

Rozważmay model SVC z dwoma różnymi rodzajami jądra linear i rbf.

In [ ]:
X, y = make_classification(random_state=0) 
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=0)
pipe = Pipeline([('scaler', StandardScaler()), ('svc', SVC(C=2.0, kernel='rbf',gamma='scale'))])
pipe.fit(X_train, y_train)
Pipeline(steps=[('scaler', StandardScaler()), ('svc', SVC())])
pipe.score(X_test, y_test)

In [ ]:
X, y = make_classification(random_state=0) 
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=0)
pipe = Pipeline([('scaler', StandardScaler()), ('svc', SVC(C=0.4, kernel='linear'))])
pipe.fit(X_train, y_train)
Pipeline(steps=[('scaler', StandardScaler()), ('svc', SVC())])
pipe.score(X_test, y_test)

Lepszy wynik modelu dostajemy dla jądra linear.

**Podstawowy bład średniokwadratowy dla różnych metod**

In [ ]:
models = []
models.append( ('Ridge', Ridge()) )
models.append( ('Lasso', Lasso()) )
models.append( ('svc-rbf', SVC(C=2.0, kernel='rbf',gamma='scale')))
models.append( ('svc-linear', SVC(C=0.4, kernel='linear')))             

In [ ]:
for name, model in models:
 pipelined_model = Pipeline([('minmax', MinMaxScaler()),('pca', PCA(n_components = 3)),(name, model)])
 pipelined_model.fit(X_train, y_train)
 y_hat = pipelined_model.predict(X_test)
 RMSE = np.sqrt(mean_squared_error(y_test, y_hat))
 print('Model: ', name, ' | RMSE: ', RMSE)
 print('----------------')
 joblib.dump(pipelined_model, '{}_model.pkl'.format(name))

**Wyregulowanie modelu**

Metoda siatki

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
logistic = LogisticRegression(solver='saga', tol=1e-2, max_iter=200,random_state=0)
distributions = dict(C=uniform(loc=0, scale=4),penalty=['l2', 'l1'])# l1 lasso l2 ridge
clf = RandomizedSearchCV(logistic, distributions, random_state=0)
search = clf.fit(X_train,y_train)
score=clf.score(X_train,y_train)
search.best_params_

In [ ]:
lr_new=LogisticRegression(C=2.195254015709299,penalty='l1', solver='liblinear')
lr_new.fit(X_train,y_train)
print("score",lr_new.score(X_test,y_test))

In [ ]:
models.append( ('RandomizedSearchCV', RandomizedSearchCV(logistic, distributions, random_state=0)) )

In [ ]:
for name, model in models:
 pipelined_model = Pipeline([('minmax', MinMaxScaler()),('pca', PCA(n_components = 3)),(name, model)])
 pipelined_model.fit(X_train, y_train)
 y_hat = pipelined_model.predict(X_test)
 RMSE = np.sqrt(mean_squared_error(y_test, y_hat))
 print('Model: ', name, ' | RMSE: ', RMSE)
 print('----------------')
 joblib.dump(pipelined_model, '{}_model.pkl'.format(name))

**Pradykcja na zbiorze testowym**

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)
model = GradientBoostingRegressor(random_state=7, n_estimators=400)
model.fit(rescaledX, y_train)
# transform the validation dataset
rescaledValidationX = scaler.transform(X_test)
predictions = model.predict(rescaledValidationX)
print(mean_squared_error(y_test, predictions))

In [ ]:
print('R^2:',metrics.r2_score(y_test, predictions))
print('Adjusted R^2:',1 - (1-metrics.r2_score(y_test, predictions))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1))
print('MAE:',metrics.mean_absolute_error(y_test, predictions))
print('MSE:',metrics.mean_squared_error(y_test, predictions))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_test, predictions)))

**Podsumowanie**

In [ ]:
models.append( ('GradientBoostingRegressor', GradientBoostingRegressor(random_state=7, n_estimators=400)) )
models.append( ('LinearRegression', LinearRegression()) )

In [ ]:
for name, model in models:
 pipelined_model = Pipeline([('minmax', MinMaxScaler()),('pca', PCA(n_components = 3)),(name, model)])
 pipelined_model.fit(X_train, y_train)
 y_hat = pipelined_model.predict(X_test)
 RMSE = np.sqrt(mean_squared_error(y_test, y_hat))
 print('Model: ', name, ' | RMSE: ', RMSE)
 print('----------------')
 joblib.dump(pipelined_model, '{}_model.pkl'.format(name))

Na podstawie wyników podstawowoego błędu średniokwadratowego widzimy, że model z najlepszym wynikiem to GradientBoostingRegressor